In [ ]:
from datetime import datetime
import os

import pandas as pd
from app.database.vector_store import VectorStore
from timescale_vector.client import uuid_from_time

# Initialize VectorStore
vec = VectorStore()

# Read the CSV file
# df = pd.read_csv("../data/faq_dataset.csv", sep=";")

# file_path = os.path.abspath("/Users/nishani/Documents/SIT723/RAG-LLM/tree-of-thought-llm/data/adax.csv")  # Move up 2 levels
file_path = os.path.abspath("/Users/nishani/Documents/SIT723/RAG-LLM/tree-of-thought-llm/data/userdata.csv")  # Move up 2 levels

print("Loading file from:", file_path)

print("Reading AdaX dataset...")
df = pd.read_csv(file_path, sep=";")
print("AdaX dataset read successfully.\n", df)
print("Preparing records for insertion...")


# Prepare data for insertion
def prepare_record(row):
    """Prepare a record for insertion into the vector store.

    This function creates a record with a UUID version 1 as the ID, which captures
    the current time or a specified time.

    Note:
        - By default, this function uses the current time for the UUID.
        - To use a specific time:
          1. Import the datetime module.
          2. Create a datetime object for your desired time.
          3. Use uuid_from_time(your_datetime) instead of uuid_from_time(datetime.now()).

        Example:
            from datetime import datetime
            specific_time = datetime(2023, 1, 1, 12, 0, 0)
            id = str(uuid_from_time(specific_time))

        This is useful when your content already has an associated datetime.
    """
    # content = f"Question: {row['question']}\nAnswer: {row['answer']}"
    # embedding = vec.get_embedding(content)
    # return pd.Series(
    #     {
    #         "id": str(uuid_from_time(datetime.now())),
    #         "metadata": {
    #             "category": row["category"],
    #             "created_at": datetime.now().isoformat(),
    #         },
    #         "contents": content,
    #         "embedding": embedding,
    #     }
    # )
    
    """
    Generates an embedding for adaptive explanations and formats the data for vector storage.
    
    Parameters:
    row (pd.Series): A row from the adaptive explanations dataset.
    vec (EmbeddingModel): An embedding model instance with a `get_embedding` method.

    Returns:
    pd.Series: A new row with an embedding and metadata.
    """
    # # Construct content based on the adapted AdaX format
    # content = (
    #     f"Task: {row['task_description']}\n"
    #     f"User State: Stress={row['stress']}, Trust={row['trust']}, Cognitive Load={row['cognitive_load']}\n"
    #     f"Game Metrics: Score={row['game_score']}, Collisions={row['num_collisions']}\n"
    #     f"Explanation (Simplified): {row['explanation_simplified']}\n"
    #     f"Explanation (Balanced): {row['explanation_balanced']}\n"
    #     f"Explanation (Step-by-Step): {row['explanation_step_by_step']}"
    # )

    # Construct content using the adapted AdaX format (previously "question;answer;category")
    
    content = (
        f"Task: {row['task_description']}\n"
        f"Best Explanation: {row['explanation_simplified']}\n"
        f"Alternative Explanation (Balanced): {row['explanation_balanced']}\n"
        f"Step-by-Step Explanation: {row['explanation_step_by_step']}"
    )
    # Generate embedding vector
    embedding =  vec.get_embedding_ollama(content)
    # embedding =  vec.get_embedding_openai(content)

    return pd.Series(
        {
            "id": str(uuid_from_time(datetime.now())),
            "metadata": {
                "stress": row["stress"],
                "trust": row["trust"],
                "cognitive_load": row["cognitive_load"],
                "game_score": row["game_score"],
                "num_collisions": row["num_collisions"],
                "created_at": datetime.now().isoformat(),
            },
            "contents": content,
            "embedding": embedding,
        }
    )


# records_df = df.apply(prepare_record, axis=1)

# # Create tables and insert data
# vec.create_tables()
# vec.create_index()  # DiskAnnIndex
# vec.upsert(records_df)


from concurrent.futures import ThreadPoolExecutor
import numpy as np

def format_content(row):
    content = (
        f"Game state: Score {row['score']}, {row['num_collisions']} collisions,\n"
        f"{row['time_left']} seconds left.\n"
        f"User state: Stress {row['stress']}, trust {row['trust']}, cognitive load {row['cognitive_load']}.\n"
        f"Task: {row['task_description']}\n"
        f"Best Explanation: {row['explanation_simplified']}\n"
        f"Alternative Explanation (Balanced): {row['explanation_balanced']}\n"
        f"Step-by-Step Explanation: {row['explanation_step_by_step']}\n"
        f"duration: {row.get('explanation_duration', 'medium')},  # Default: 'medium'\n"
        f"granularity: {row.get('explanation_granularity', 'steps')},  # Default: 'steps'\n"
        f"timing: {row.get('explanation_timing', 'reactive')}  # Default: 'reactive'"
    )
    return content

def process_row(row):
     
    # text_data = f"Game state: Score {row['score']}, {row['num_collisions']} collisions, {row['time_left']} seconds left. User state: Stress {row['stress']}, trust {row['trust']}, cognitive load {row['cognitive_load']}."
    contents = format_content(row)
    embedding = vec.get_embedding_ollama(contents)

    print("Embedding shape:", np.array(embedding).shape)  # Debugging step
    assert len(embedding) == 1024, f"Embedding size mismatch! Expected 1024, got {len(embedding)}"

    return pd.Series(
        {
            "id": str(uuid_from_time(datetime.now())),
            "metadata": {
                "stress": row["stress"],
                "trust": row["trust"],
                "cognitive_load": row["cognitive_load"],
                "game_score": row["score"],
                "num_collisions": row["num_collisions"],
                "created_at": datetime.now().isoformat(),
            },
            "contents": contents,
            "embedding": embedding,
        }
    )

records_df = df.apply(process_row, axis=1)
vec.create_tables()
vec.create_index()  # DiskAnnIndex
vec.upsert(records_df)

# with ThreadPoolExecutor(max_workers=4) as executor:
#     embeddings = list(executor.map(process_row, df))

/Users/nishani/anaconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 00:11:26,471 - INFO - Use pytorch device_name: mps
2025-03-10 00:11:26,471 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


Loading file from: /Users/nishani/Documents/SIT723/RAG-LLM/tree-of-thought-llm/data/userdata.csv
Reading AdaX dataset...
AdaX dataset read successfully.
                     timestamp  score  num_collisions  time_left  stress  \
0  2025-03-04T08:18:58.376354     19               3          4  medium   
1  2025-03-04T08:18:59.376354     15              10         57    high   
2  2025-03-04T08:19:00.376354     21               1         98    high   
3  2025-03-04T08:19:01.376354     50               0         22  medium   
4  2025-03-04T08:19:02.376354     45               5         34  medium   

    trust cognitive_load                                   task_description  \
0     low         medium         AI picked up a dish to complete the order.   
1    high           high        AI moved to chop station to prepare onions.   
2     low            low  AI placed an onion in the pot as one was left ...   
3     low           high  AI delivered a completed dish to the serving s...   


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.77it/s]
2025-03-10 00:11:32,766 - INFO - Embedding generated in 0.151 seconds
2025-03-10 00:11:32,768 - INFO - Use pytorch device_name: mps
2025-03-10 00:11:32,768 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


Embedding shape: (1024,)


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.51it/s]
2025-03-10 00:11:38,015 - INFO - Embedding generated in 0.083 seconds
2025-03-10 00:11:38,016 - INFO - Use pytorch device_name: mps
2025-03-10 00:11:38,017 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


...

Embedding shape: (1024,)


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.48it/s]
2025-03-10 00:11:46,842 - INFO - Embedding generated in 0.109 seconds
2025-03-10 00:11:46,843 - INFO - Use pytorch device_name: mps
2025-03-10 00:11:46,844 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


Embedding shape: (1024,)


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.45it/s]
2025-03-10 00:11:52,236 - INFO - Embedding generated in 0.122 seconds
2025-03-10 00:11:52,317 - INFO - Inserted 5 records into adax_embeddings


Embedding shape: (1024,)
[('058f4dd8-fce8-11ef-9895-795ee886a8f8', {'stress': 'medium', 'trust': 'low', 'cognitive_load': 'medium', 'game_score': 19, 'num_collisions': 3, 'created_at': '2025-03-10T00:11:32.766619'}, "Game state: Score 19, 3 collisions,\n4 seconds left.\nUser state: Stress medium, trust low, cognitive load medium.\nTask: AI picked up a dish to complete the order.\nBest Explanation: Picked dish for final order step.\nAlternative Explanation (Balanced): AI identified the final step in the order and picked up the dish to deliver it efficiently.\nStep-by-Step Explanation: 1. Checked order queue. 2. Picked the dish. 3. Moved toward delivery station. 4. Delivered the dish.\nduration: short,  # Default: 'medium'\ngranularity: steps,  # Default: 'steps'\ntiming: reactive  # Default: 'reactive'", list([0.024960938841104507, -0.05730147659778595, 0.015773896127939224, -0.009160668589174747, -0.03261959180235863, -0.0034374415408819914, -0.03249948099255562, -0.023833131417632103,

In [ ]:
import argparse
from tot.methods.bfs import solve
from tot.tasks.game24 import Game24Task
from tot.tasks.text import TextTask
from tot.tasks.adax import AdaXTask


# args = argparse.Namespace(backend='gpt-4', temperature=0.7, task='game24', naive_run=False, prompt_sample=None, method_generate='propose', method_evaluate='value', method_select='greedy', n_generate_sample=1, n_evaluate_sample=3, n_select_sample=5)

# task = Game24Task()
# ys, infos = solve(args, task, 900)
# print(ys[0])

# args = argparse.Namespace(backend='gpt-4', temperature=0.8, task='adax', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)
args = argparse.Namespace(backend='mistral:latest', temperature=0.7, task='adax', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)

task = AdaXTask()
ys, infos = solve(args, task, 0)
print(ys[0])


# args = argparse.Namespace(backend='gpt-4', temperature=1.0, task='text', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)

# task = TextTask()
# ys, infos = solve(args, task, 0)
# print(ys[0])

2025-03-10 00:12:02,142 - INFO - Use pytorch device_name: mps
2025-03-10 00:12:02,143 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2
Batches: 100%|██████████| 1/1 [00:00<00:00, 13.90it/s]
2025-03-10 00:12:06,746 - INFO - Embedding generated in 0.075 seconds
2025-03-10 00:12:06,775 - INFO - Vector search completed in 0.029 seconds


Query text:  Understanding AI's reasoning to improve trust
Query embedding:  1024
   stress   trust cognitive_load  game_score  num_collisions  \
0  medium     low           high          85               0   
1  medium     low         medium          19               3   
2  medium  medium         medium          45               5   
3    high     low            low          21               1   

                                task_description  \
0  Understanding AI's reasoning to improve trust   
1                                            NaN   
2                                            NaN   
3                                            NaN   

                                     id  \
0                                   NaN   
1  058f4dd8-fce8-11ef-9895-795ee886a8f8   
2  112a4c56-fce8-11ef-abb3-c992f7f4e08a   
3  0b412742-fce8-11ef-a497-46f1f1cdc94f   

                                             content  \
0                                                NaN   
1  Game 

2025-03-10 00:12:14,136 - INFO - Use pytorch device_name: mps
2025-03-10 00:12:14,137 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


<Response [200]>
ANSWER: 
Added onion to complete soup quickly.

Thought process:
- Detected incomplete soup (User state: Stress high, trust low, cognitive load low)
- Used short and simple phrasing (10 words)
\XAI Features Used:
- short
- simple

Context: True


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.63it/s]
2025-03-10 00:12:20,418 - INFO - Embedding generated in 0.107 seconds
2025-03-10 00:12:20,429 - INFO - Vector search completed in 0.011 seconds


...


-- new_ys --: (' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI concentrates on dishes almost done due to time constraint. Simplify workload for reduced stress.', ' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI concentrates on dishes close to finish (4s left) to reduce your stress.', ' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI focusing on dishes close to finish due to time constraints. Simplify actions for your medium stress.', ' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI focuses on tasks nearing completion. Simplify actions to

Batches: 100%|██████████| 1/1 [00:00<00:00, 24.78it/s]
2025-03-10 00:13:16,134 - INFO - Embedding generated in 0.044 seconds
2025-03-10 00:13:16,142 - INFO - Vector search completed in 0.008 seconds


Query text:  Understanding AI's latest decision
Query embedding:  1024
   stress   trust cognitive_load  game_score  num_collisions  \
0                                          0               0   
1  medium     low         medium          19               3   
2  medium  medium         medium          45               5   
3    high     low            low          21               1   

                     task_description                                    id  \
0  Understanding AI's latest decision                                   NaN   
1                                 NaN  058f4dd8-fce8-11ef-9895-795ee886a8f8   
2                                 NaN  112a4c56-fce8-11ef-abb3-c992f7f4e08a   
3                                 NaN  0b412742-fce8-11ef-a497-46f1f1cdc94f   

                                             content  \
0                                                NaN   
1  Game state: Score 19, 3 collisions,\n4 seconds...   
2  Game state: Score 45, 5 collisions,\n34 s

2025-03-10 00:13:18,539 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:18,868 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:19,192 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:19,531 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:19,773 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:22,513 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:23,974 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:26,025 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:27,366 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 00:13:29,099 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/cha


-- new_ys --: (' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI concentrates on dishes almost done due to time constraint. Simplify workload for reduced stress. AI focuses on tasks nearing completion.\n(Explanation reasoning: High cognitive load and medium stress levels necessitate simplified essential reasoning.)', ' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI concentrates on dishes almost done due to time constraint. Simplify workload for reduced stress. AI focuses on tasks nearing completion, reducing your stress.', ' AI is prioritizing a dish with less time remaining (4 seconds). Your stress level is medium, so aim for simplicity: "AI focuses on tasks nearing completion." AI concentrates on dishes almost done due to time constraint. Simplify workload for

In [ ]:
import argparse
from tot.methods.bfs import solve
from tot.tasks.game24 import Game24Task
from tot.tasks.text import TextTask
from tot.tasks.adax import AdaXTask


# args = argparse.Namespace(backend='gpt-4', temperature=0.7, task='game24', naive_run=False, prompt_sample=None, method_generate='propose', method_evaluate='value', method_select='greedy', n_generate_sample=1, n_evaluate_sample=3, n_select_sample=5)

# task = Game24Task()
# ys, infos = solve(args, task, 900)
# print(ys[0])

# args = argparse.Namespace(backend='gpt-4', temperature=0.8, task='adax', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)
args = argparse.Namespace(backend='mistral:latest', temperature=0.7, task='adax', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)

task = AdaXTask()
ys, infos = solve(args, task, 0)
print(ys[0])


# args = argparse.Namespace(backend='gpt-4', temperature=1.0, task='text', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)

# task = TextTask()
# ys, infos = solve(args, task, 0)
# print(ys[0])

2025-03-10 11:15:36,545 - INFO - Use pytorch device_name: mps
2025-03-10 11:15:36,546 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


functools.partial(<function mistral_local at 0x31037f0a0>, model='mistral:latest', temperature=0.7)


Batches: 100%|██████████| 1/1 [00:00<00:00, 23.77it/s]
2025-03-10 11:15:52,089 - INFO - Embedding generated in 0.046 seconds
2025-03-10 11:15:52,097 - INFO - Vector search completed in 0.009 seconds


Query text:  Understanding AI's latest decision
Query embedding:  1024
   stress   trust cognitive_load  game_score  num_collisions  \
0                                          0               0   
1  medium     low         medium          19               3   
2  medium  medium         medium          45               5   
3    high     low            low          21               1   

                     task_description                                    id  \
0  Understanding AI's latest decision                                   NaN   
1                                 NaN  058f4dd8-fce8-11ef-9895-795ee886a8f8   
2                                 NaN  112a4c56-fce8-11ef-abb3-c992f7f4e08a   
3                                 NaN  0b412742-fce8-11ef-a497-46f1f1cdc94f   

                                             content  \
0                                                NaN   
1  Game state: Score 19, 3 collisions,\n4 seconds...   
2  Game state: Score 45, 5 collisions,\n34 s

2025-03-10 11:15:57,075 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:15:57,887 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:15:58,238 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:15:59,622 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:00,022 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:03,182 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:05,052 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:07,584 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:09,269 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:11,095 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/cha

...


-- new_ys --: (" AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load and medium stress. It's aiming for fewer collisions to build trust. AI focuses on completing tasks quickly and reducing collisions to improve trust.", " AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load and medium stress. It's aiming for fewer collisions to build trust. AI focuses on completing tasks efficiently (19 points, 3 collisions) amidst high cognitive load & medium stress. Minimizing collisions aims to boost trust.", " AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load and medium stress. It's aiming for fewer collisions to build trust. AI is focusing on task completion (score 19, 3 collisions, 4 sec) to reduce stress and improve efficiency. It aims for fewer collisions to build trust.", " AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load a

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.56it/s]
2025-03-10 11:16:38,299 - INFO - Embedding generated in 0.052 seconds
2025-03-10 11:16:38,309 - INFO - Vector search completed in 0.010 seconds


Query text:  Understanding AI's latest decision
Query embedding:  1024
   stress   trust cognitive_load  game_score  num_collisions  \
0                                          0               0   
1  medium     low         medium          19               3   
2  medium  medium         medium          45               5   
3    high     low            low          21               1   

                     task_description                                    id  \
0  Understanding AI's latest decision                                   NaN   
1                                 NaN  058f4dd8-fce8-11ef-9895-795ee886a8f8   
2                                 NaN  112a4c56-fce8-11ef-abb3-c992f7f4e08a   
3                                 NaN  0b412742-fce8-11ef-a497-46f1f1cdc94f   

                                             content  \
0                                                NaN   
1  Game state: Score 19, 3 collisions,\n4 seconds...   
2  Game state: Score 45, 5 collisions,\n34 s

2025-03-10 11:16:40,669 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:41,116 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:41,617 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:42,286 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:43,136 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:46,344 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:48,792 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:51,488 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:53,974 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 11:16:56,405 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/cha


-- new_ys --: (" AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load and medium stress. It's aiming for fewer collisions to build trust. AI focuses on completing tasks quickly and reducing collisions to improve trust. AI is speeding up task completion (Score: 19, Collisions: 3, Time left: 4) due to high cognitive load and medium stress. It aims for fewer collisions to boost trust.", " AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load and medium stress. It's aiming for fewer collisions to build trust. AI focuses on completing tasks quickly and reducing collisions to improve trust. AI aims for quick completions, fewer collisions to build trust. High cognitive load, prioritizing task completion.", " AI is prioritizing task completion (score 19, 3 collisions, 4 sec) due to high cognitive load and medium stress. It's aiming for fewer collisions to build trust. AI focuses on completing tasks quickly and re

In [ ]:
import argparse
from tot.methods.bfs import solve
from tot.tasks.game24 import Game24Task
from tot.tasks.text import TextTask
from tot.tasks.adax import AdaXTask


# args = argparse.Namespace(backend='gpt-4', temperature=0.7, task='game24', naive_run=False, prompt_sample=None, method_generate='propose', method_evaluate='value', method_select='greedy', n_generate_sample=1, n_evaluate_sample=3, n_select_sample=5)

# task = Game24Task()
# ys, infos = solve(args, task, 900)
# print(ys[0])

# args = argparse.Namespace(backend='gpt-4', temperature=0.8, task='adax', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)
args = argparse.Namespace(backend='mistral:latest', temperature=0.7, task='adax', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=3, n_evaluate_sample=3, n_select_sample=1)

task = AdaXTask()
ys, infos = solve(args, task, 0)
print(ys[0])


# args = argparse.Namespace(backend='gpt-4', temperature=1.0, task='text', naive_run=False, prompt_sample="cot", method_generate='sample', method_evaluate='vote', method_select='greedy', n_generate_sample=5, n_evaluate_sample=5, n_select_sample=1)

# task = TextTask()
# ys, infos = solve(args, task, 0)
# print(ys[0])

2025-03-10 12:46:26,572 - INFO - Use pytorch device_name: mps
2025-03-10 12:46:26,572 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


functools.partial(<function mistral_local at 0x31037f0a0>, model='mistral:latest', temperature=0.7)


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.29it/s]
2025-03-10 12:46:33,030 - INFO - Embedding generated in 0.069 seconds
2025-03-10 12:46:33,040 - INFO - Vector search completed in 0.010 seconds


Query text:  Understanding AI's latest decision
Query embedding:  1024
   stress   trust cognitive_load  game_score  num_collisions  \
0                                          0               0   
1  medium     low         medium          19               3   
2  medium  medium         medium          45               5   
3    high     low            low          21               1   

                     task_description                                    id  \
0  Understanding AI's latest decision                                   NaN   
1                                 NaN  058f4dd8-fce8-11ef-9895-795ee886a8f8   
2                                 NaN  112a4c56-fce8-11ef-abb3-c992f7f4e08a   
3                                 NaN  0b412742-fce8-11ef-a497-46f1f1cdc94f   

                                             content  \
0                                                NaN   
1  Game state: Score 19, 3 collisions,\n4 seconds...   
2  Game state: Score 45, 5 collisions,\n34 s

2025-03-10 12:46:38,117 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:46:38,492 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:46:38,964 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:46:41,652 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:46:43,219 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:46:44,587 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:46:44,595 - INFO - Use pytorch device_name: mps
2025-03-10 12:46:44,596 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


...


-- new_ys --: (' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef prioritizes speed and accuracy to reduce collisions. (Adjusted for medium cognitive load, low trust)', ' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef is focusing on reducing collisions to ease user stress.', ' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef prioritizes fewer collisions to reduce mistakes.')
-- sol values --: (2, 1, 0)
-- choices --: [' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef prioritizes speed and accuracy to reduce collisions. (Adjusted for medium cognitive load, low trust)']



Batches: 100%|██████████| 1/1 [00:00<00:00, 23.51it/s]
2025-03-10 12:47:02,726 - INFO - Embedding generated in 0.046 seconds
2025-03-10 12:47:02,736 - INFO - Vector search completed in 0.009 seconds


Query text:  Understanding AI's latest decision
Query embedding:  1024
   stress   trust cognitive_load  game_score  num_collisions  \
0                                          0               0   
1  medium     low         medium          19               3   
2  medium  medium         medium          45               5   
3    high     low            low          21               1   

                     task_description                                    id  \
0  Understanding AI's latest decision                                   NaN   
1                                 NaN  058f4dd8-fce8-11ef-9895-795ee886a8f8   
2                                 NaN  112a4c56-fce8-11ef-abb3-c992f7f4e08a   
3                                 NaN  0b412742-fce8-11ef-a497-46f1f1cdc94f   

                                             content  \
0                                                NaN   
1  Game state: Score 19, 3 collisions,\n4 seconds...   
2  Game state: Score 45, 5 collisions,\n34 s

2025-03-10 12:47:05,032 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:47:05,867 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:47:06,333 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:47:08,326 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:47:10,094 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-10 12:47:12,162 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



-- new_ys --: (' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef prioritizes speed and accuracy to reduce collisions. (Adjusted for medium cognitive load, low trust) "AI chef streamlines workflow to minimize collisions, lower stress."\n(Adjusted for medium cognitive load, low trust) The AI chef is working efficiently to prevent collisions, helping you manage stress levels.', ' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef prioritizes speed and accuracy to reduce collisions. (Adjusted for medium cognitive load, low trust) AI chef optimizes actions for efficiency, minimizing collisions (low trust, medium cognitive load).', ' "AI chef prepares food efficiently, aiming for fewer collisions." (Adjusted for medium cognitive load, low trust) AI chef prioritizes speed and accuracy to reduce collisions. (Adjusted for medium cognitive load, low tr